<div class='alert alert-block alert-info'>Ophalen belasting-waarden vanuit een Waterinfo service</div>

In [ ]:
# importeer de benodigde bibliotheken
import os
from pathlib import Path

# voor het plotten van de data
import matplotlib.pyplot as plt
import numpy as np

# Hiermee kan je markdown printen tijdens uitvoeren van functie
from IPython.display import Markdown, display

from toolbox_continu_inzicht import datetime_from_epoch

# Vanuit toolbox gebruiken we standaard 'Config' en 'DataAdapter'.
# De functie die we willen gebruiken om data op te halen is 'LoadsWaterinfo'
from toolbox_continu_inzicht.base.config import Config
from toolbox_continu_inzicht.base.data_adapter import DataAdapter
from toolbox_continu_inzicht.loads import LoadsWaterinfo

**loads_waterinfo_config.yaml:**
```Yaml
GlobalVariables:
    rootdir: "data_sets"   
    moments: [-24,0,24,48]

    csv:
        sep: ","

    LoadsWaterinfo:
        datatype: "waterhoogte"
        observedhours: 48
        predictionhours: 48
        momentsupdate: True
        MISSING_VALUE: -9999.0

DataAdapter: 
    locaties: 
        type: csv
        path: "locations_waterinfo.csv"
    waterstanden: 
        type: csv
        path: "waterstanden.csv"
```

**locations_waterinfo.csv:**
```csv
id,name,code
1,Rottedamse hoek,Rotterdamse-hoek(FL02)-1
2,Dordrecht,Dordrecht(DORD)
3,Moerdijk,Moerdijk(MOER)
4,Pannerdense kop,Pannerdense-kop(PANN)
5,IJsselkop,IJsselkop(IJSS)
6,Den helder,Den-Helder(DENH)
```

In [ ]:
# Path naar config bestand
config_file = Path("data_sets/loads_waterinfo_config.yaml")
if os.path.exists(config_file):
    # Maak config aan, deze gebruiken we om de adapter aan te maken
    display(
        Markdown("*Maak config aan, deze gebruiken we om de adapter aan te maken.*")
    )
    config = Config(config_path=config_file)
    config.lees_config()

    display(Markdown("*Maak DataAdapter aan, deze gebruiken bij de functie.*"))
    data_adapter = DataAdapter(config=config)
else:
    display(Markdown(f"**⚠️ Config file '{config_file}' not found.**"))

In [ ]:
# Oude gegevens verwijderen
display(Markdown("Oude gegevens verwijderen"))

output_info = config.data_adapters
output_file = Path(
    config.global_variables["rootdir"] / Path(output_info["waterstanden"]["path"])
)
if os.path.exists(output_file):
    os.remove(output_file)

In [ ]:
# Gebruik de 'LoadsWaterinfo' om gegevens uit Waterinfo te halen
display(Markdown("Gebruik de 'LoadsWaterinfo' om gegevens uit Waterinfo te halen"))
waterinfo = LoadsWaterinfo(
    data_adapter=data_adapter, input="locaties", output="waterstanden"
)

display(Markdown("Voer de functie uit."))
df_out = waterinfo.run()

In [ ]:
# test of we data hebben opgehaald
if os.path.exists(output_file):
    display(Markdown(f"*{output_file} is aangemaakt.*"))
else:
    display(Markdown(f"*{output_file} is niet aangemaakt.*"))

# controleer de opgehaalde waarden
display(Markdown("**Controleer de opgehaalde waarden**"))
print(waterinfo.df_out)

In [ ]:
# split de dataframe in tweeën & process
values_nan = df_out[df_out["value"] == 999999999.0].index
for val in values_nan:
    df_out.loc[val, "value"] = np.nan
df_out.sort_index()

df_plot = df_out.set_index(df_out["datetime"].apply(lambda x: datetime_from_epoch(x)))
df_plot = df_plot[df_plot["objectid"] == 2]

measurement_index = df_plot["parameterid"] == 1
forecast_index = df_plot["parameterid"] == 99
df_plot_measurements = df_plot[measurement_index][["value"]]
df_plot_forecast = df_plot[forecast_index][["value"]]

In [ ]:
df_plot_measurements

In [ ]:
fig, ax = plt.subplots()
df_plot_measurements.plot(color="C0", ax=ax)
df_plot_forecast.plot(color="C1", ax=ax)
ax.legend(["Prediction", "Measurement"])